Brief MongoDB
===========

- Connecter la base MongoDB

In [ ]:
# prompt: Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")


- On va utiliser la base de données listingsAndReviews

1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

In [ ]:
# prompt: Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()
price = first_doc["price"]
print(price)


2. Récupérer les frais de ménage du premier document dans la collection listingsAndReviews.


In [ ]:
# prompt: Récupérer les frais de ménage du premier document dans la collection listingsAndReviews.

import pymongo
# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()
cleaning_fee = first_doc["cleaning_fee"]
print(cleaning_fee)

# 2. Récupérer les frais de ménage du premier document dans la collection listingsAndReviews.
#


3. Trouver le nom de l'hôte, l'emplacement de l'hôte et les informations sur l'hôte du premier document dans la collection listingsAndReviews.


In [ ]:
# prompt: Trouver le nom de l'hôte, l'emplacement de l'hôte et les informations sur l'hôte du premier document dans la collection listingsAndReviews.

import pymongo
# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()
host_name = first_doc["host"]["host_name"]
host_location = first_doc["host"]["host_location"]
host_about = first_doc["host"]["host_about"]
print("Host Name:", host_name)
print("Host Location:", host_location)
print("Host About:", host_about)

# 2. Récupérer les frais de ménage du premier document dans la collection listingsAndReviews.
#

# 3. Trouver le nom de l'hôte, l'emplacement de l'hôte et les informations sur l'hôte du premier document dans la collection listingsAndReviews.
#


4. Récupérer le nombre de chambres du premier document dans la collection listingsAndReviews.

In [ ]:
# prompt:    Récupérer le nombre de chambres du premier document dans la collection listingsAndReviews.

import pymongo
# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()
bedrooms = first_doc["bedrooms"]
print(bedrooms)

# 2. Récupérer les frais de ménage du premier document dans la collection listingsAndReviews.
#

# 3. Trouver le nom de l'hôte, l'emplacement de l'hôte et les informations sur l'hôte du premier document dans la collection listingsAndReviews.
#

# 4. Récupérer le nombre de chambres du premier document dans la collection listingsAndReviews.


5. Récupérer le nombre d'invités inclus dans le premier document de la collection listingsAndReviews.

In [ ]:
# prompt:     Récupérer le nombre d'invités inclus dans le premier document de la collection listingsAndReviews.

import pymongo
# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()
accomodates = first_doc["accommodates"]
print(accomodates)

# 2. Récupérer les frais de ménage du premier document dans la collection listingsAndReviews.
#

# 3. Trouver le nom de l'hôte, l'emplacement de l'hôte et les informations sur l'hôte du premier document dans la collection listingsAndReviews.
#

# 4. Récupérer le nombre de chambres du premier document dans la collection listingsAndReviews.

# 5. Récupérer le nombre d'invités inclus dans le premier document de la collection listingsAndReviews.


6. Écrire une requête MongoDB pour vérifier si l'hôte a une photo de profil dans le premier document de la collection listingsAndReviews.

In [ ]:
# prompt: Écrire une requête MongoDB pour vérifier si l'hôte a une photo de profil dans le premier document de la collection listingsAndReviews.

import pymongo

# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()

# Vérifier si l'hôte a une photo de profil
has_profile_pic = "host_picture_url" in first_doc["host"]

print(has_profile_pic)


7. Écrire une requête MongoDB pour vérifier si l'identité de l'hôte a été vérifiée dans le premier document de la collection listingsAndReviews.

In [ ]:
# prompt: crire une requête MongoDB pour vérifier si l'identité de l'hôte a été vérifiée dans le premier document de la collection listingsAndReviews.

# Vérifier si l'identité de l'hôte a été vérifiée
is_identity_verified = first_doc["host"]["host_identity_verified"]

print(is_identity_verified)


8. Écrire une requête MongoDB pour trouver combien de listings l'hôte possède dans le premier document de la collection listingsAndReviews.

In [ ]:
# prompt: Écrire une requête MongoDB pour trouver combien de listings l'hôte possède dans le premier document de la collection listingsAndReviews.

import pymongo

# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()

# Trouver combien de listings l'hôte possède
num_listings = first_doc["host"]["host_listings_count"]

print(num_listings)


9. Écrire une requête MongoDB pour trouver l'adresse de la rue du premier document dans la collection listingsAndReviews.


In [ ]:
# prompt: Écrire une requête MongoDB pour trouver l'adresse de la rue du premier document dans la collection listingsAndReviews.

# Brief MongoDB
# ===========
# - Connecter la base MongoDB

# Brief MongoDB
# ===========
# - Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# - On va utiliser la base de données listingsAndReviews
# 1. Trouver le prix par nuit du premier document dans la collection listingsAndReviews.

db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]
first_doc = collection.find_one()

# Trouver l'adresse de la rue
street_address = first_doc["address"]["street"]

print(street_address)


10. Trouver tous les listings dans la collection listingsAndReviews où le champ property_type est défini sur "House".


In [ ]:
# prompt: Trouver tous les listings dans la collection listingsAndReviews où le champ property_type est défini sur "House".

import pymongo

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings où property_type est "House"
house_listings = collection.find({"property_type": "House"})

# Afficher les résultats
for listing in house_listings:
  print(listing)


11. Trouver tous les listings dans la collection listingsAndReviews avec listing_url, nom, nom de l'hôte, emplacement de l'hôte et prix qui ont un prix par nuit supérieur à 500 $.

In [ ]:
# prompt: Trouver tous les listings dans la collection listingsAndReviews avec listing_url, nom, nom de l'hôte, emplacement de l'hôte et prix qui ont un prix par nuit supérieur à 500 $.

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings avec un prix par nuit supérieur à 500 $
expensive_listings = collection.find(
    {"price": {"$gt": "500"}},
    {"listing_url": 1, "name": 1, "host.host_name": 1, "host.host_location": 1, "price": 1, "_id": 0}
)

# Afficher les résultats
for listing in expensive_listings:
  print(listing)



12. Trouver tous les listings dans la collection listingsAndReviews situés au Brésil et ayant une note d'évaluation d'au moins 9. Retourner le nom, l'adresse et la note d'évaluation.

In [ ]:
# prompt: Trouver tous les listings dans la collection listingsAndReviews situés au Brésil et ayant une note d'évaluation d'au moins 9. Retourner le nom, l'adresse et la note d'évaluation.

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings au Brésil avec une note d'évaluation d'au moins 9
brazilian_listings = collection.find(
    {
        "address.country": "Brazil",
        "review_scores.review_scores_rating": {"$gte": 9}
    },
    {
        "name": 1,
        "address": 1,
        "review_scores.review_scores_rating": 1,
        "_id": 0
    }
)

# Afficher les résultats
for listing in brazilian_listings:
  print(listing)


13. Trouver tous les listings avec nom, adresse, hôte dans la collection listingsAndReviews qui ont un hôte avec un nom d'hôte contenant le mot "Livia"

In [ ]:
# prompt: Trouver tous les listings avec nom, adresse, hôte dans la collection listingsAndReviews qui ont un hôte avec un nom d'hôte contenant le mot "Livia"

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings avec un nom d'hôte contenant "Livia"
livia_listings = collection.find(
    {"host.host_name": {"$regex": "Livia"}},
    {"name": 1, "address": 1, "host": 1, "_id": 0}
)

# Afficher les résultats
for listing in livia_listings:
  print(listing)


14. Récupérer tous les documents avec listing_url, nom, commodité, hôte dans la collection listingsAndReviews où le tableau amenities contient à la fois "TV" et "Wifi".

In [ ]:
# prompt: Récupérer tous les documents avec listing_url, nom, commodité, hôte dans la collection listingsAndReviews où le tableau amenities contient à la fois "TV" et "Wifi".

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings avec "TV" et "Wifi" dans les commodités
tv_wifi_listings = collection.find(
    {"amenities": {"$all": ["TV", "Wifi"]}},
    {"listing_url": 1, "name": 1, "amenities": 1, "host": 1, "_id": 0}
)

# Afficher les résultats
for listing in tv_wifi_listings:
  print(listing)


15. Trouver tous les listings avec listing_url, nom, adresse, vérifications de l'hôte et nombre des vérifications de l'hôte dans la collection listingsAndReviews qui ont un hôte avec au moins 3 vérifications.

In [ ]:
# prompt: Trouver tous les listings avec listing_url, nom, adresse, vérifications de l'hôte et nombre des vérifications de l'hôte dans la collection listingsAndReviews qui ont un hôte avec au moins 3 vérifications.

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings avec un hôte ayant au moins 3 vérifications
verified_listings = collection.find(
    {"host.host_verifications": {"$size": {"$gte": 3}}},
    {
        "listing_url": 1,
        "name": 1,
        "address": 1,
        "host.host_verifications": 1,
        "host.host_total_listings_count": 1,
        "_id": 0
    }
)

# Afficher les résultats
for listing in verified_listings:
  print(listing)


16. Trouver tous les listings avec listing_url, nom, adresse, host_picture_url dans la collection listingsAndReviews qui ont un hôte avec une URL de photo.

In [ ]:
# prompt: Trouver tous les listings avec listing_url, nom, adresse, host_picture_url dans la collection listingsAndReviews qui ont un hôte avec une URL de photo.

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Utiliser la base de données listingsAndReviews
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

# Trouver tous les listings avec un hôte ayant une URL de photo
listings_with_host_photo = collection.find(
    {"host.host_picture_url": {"$exists": True}},
    {
        "listing_url": 1,
        "name": 1,
        "address": 1,
        "host.host_picture_url": 1,
        "_id": 0
    }
)

# Afficher les résultats
for listing in listings_with_host_photo:
  print(listing)


Créer une API flask en définissant la  route suivante pour trouver les documents contenant les amenities dans le marché spécifié et retourner les ``listings_url``:
- ``/amenities`` prend un json:


```
{
    city: string
    amenties: amenties_list
}
```


In [ ]:
# prompt: Créer une API flask en définissant la route suivante pour trouver les documents contenant les amenities dans le marché spécifié et retourner les listings_url:
#     /amenities prend un json:
# {
#     city: string
#     amenties: amenties_list
# }

from flask import Flask, request, jsonify
import pymongo

app = Flask(__name__)

# Connecter la base MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["listingsAndReviews"]
collection = db["listingsAndReviews"]

@app.route('/amenities', methods=['POST'])
def find_listings_with_amenities():
  data = request.get_json()
  city = data.get('city')
  amenities = data.get('amenities')

  # Trouver tous les listings dans la ville spécifiée avec les commodités spécifiées
  listings = collection.find(
    {
      "address.market": city,
      "amenities": {"$all": amenities}
    },
    {"listing_url": 1, "_id": 0}
  )

  # Extraire les URLs des listings
  listing_urls = [listing["listing_url"] for listing in listings]

  return jsonify(listing_urls)

if __name__ == '__main__':
  app.run(debug=True)
